In [3]:
# For NLP
import nltk
# natural language toolkit for human response understanding
nltk.download('punkt') #for tokenization of sentences into words
from nltk.stem.lancaster import LancasterStemmer  #used in reducing the different form of words into single base word
#like cook, cooked, cooking into base/stem word: cook
stemmer=LancasterStemmer()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 KB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=b07bf607e85f7c06379aeb43e93bd4ab9aee10e09cd123aa34e8bc3cb5baf7bf
  Stored in directory: /root/.cache/pip/wheels/65/9b/15/cb1e6b279c14ed897530d15cfd7da8e3df8a947e593f5cfe59
Successfully built tflearn


In [5]:
# Tensorflow processing
import tensorflow as tf
import numpy as np  
import tflearn #for different layers in deep neural netwrok
import random #to generate the numbers
import json #to read the intent json file and train the model

Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{"intents": [\r\n        {"tag": "greeting",\r\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day"],\r\n         "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "goodbye",\r\n         "patterns": ["Bye", "See you later", "Goodbye"],\r\n         "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]\r\n        },\r\n        {"tag": "thanks",\r\n         "patterns": ["Thanks", "Thank you", "That\'s helpful"],\r\n         "responses": ["Happy to help!", "Any time!", "My pleasure"]\r\n        },\r\n        {"tag": "hours",\r\n         "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],\r\n         "responses": ["We\'re open every day 9am-9pm", "Our hours are 9am-9pm every day"]\r\n        },\r\n        {"tag": "location",\r\n         "patterns": 

In [7]:
# importing the chatbot intents
with open('intents.json') as json_data:
  intents=json.load(json_data)

In [8]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'hours',
   'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of Lon

In [9]:
words=[] #storing individual words
classes=[] #storing the tags/classes of intent
documents=[] 
ignore=['?','!'] #punctuation/speacial marks to be excluded from words

for intent in intents['intents']:
  for pattern in intent['patterns']:
    # tokenization of each word in every sentence/pattern of patterns
    w=nltk.word_tokenize(pattern)
    words.extend(w) #adding individual words in empty list 
    documents.append((w,intent['tag'])) #adding words in documents for each tags
    # adding each tags in classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [10]:
# converting into lowercase and removing duplicates and stemming of words
words=[stemmer.stem(w.lower()) for w in words if w not in ignore] #converts in lowercase+stemming+ignoring the ignore words
words=sorted(list(set(words))) #removing the duplicate words by set operation & storing alphabetically

In [11]:
# removing duplicate classes
classes=sorted(list(set(classes)))

print(len(documents),"documents")
print(len(classes),"classses",classes)
print(len(words),"unique stemmed words",words)

31 documents
9 classses ['deliveryoption', 'goodbye', 'greeting', 'hours', 'location', 'menu', 'payments', 'thanks', 'todaysmenu']
57 unique stemmed words ["'s", 'acceiv', 'address', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'cours', 'credit', 'day', 'del', 'delicy', 'delivery', 'dish', 'do', 'food', 'for', 'from', 'good', 'goodby', 'hello', 'help', 'hi', 'hom', 'hour', 'how', 'is', 'lat', 'loc', 'main', 'mastercard', 'me', 'menu', 'most', 'on', 'op', 'opt', 'provid', 'resta', 'see', 'serv', 'situ', 'spec', 'tak', 'tel', 'thank', 'that', 'the', 'ther', 'today', 'what', 'when', 'wher', 'yo', 'you']


In [12]:
# creating the training data set
training=[]
output=[]
# create empty array for output
output_empty=[0]*len(classes)

# creating training set, bag of words for each sentence
for doc in documents:
  # for bag of words array
  bag=[]
  # list of tokenized words
  pattern_words=doc[0]
  # stemming each word
  pattern_words=[stemmer.stem(word.lower()) for word in pattern_words]
  # for current tag append 1 and for rest append 0 below 
  # by matching the patternword with words array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
  
  # for current tag output is 1 and 0 for rest other tags
  output_row=list(output_empty)
  output_row[classes.index(doc[1])]=1

  training.append([bag,output_row])

# shuffling features and turning into np.array
random.shuffle(training)
training=np.array(training)

# creating training lists
# features(intent patterns)
trainx=list(training[:,0])
# labels(associated tags or labelled class)
trainy=list(training[:,1])

<ipython-input-12-ed40cee26c2a>:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


In [13]:
# restting underlying graph data
tf.compat.v1.reset_default_graph()

# building the neural network
# input layer
net=tflearn.input_data(shape=[None,len(trainx[0])])
# 2 hidden layers
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,10)
# ouptut layer having trainy[0] number of fetaures
net=tflearn.fully_connected(net,len(trainy[0]),activation="softmax")
# applying the regression on the input
net=tflearn.regression(net)


# dnn model
model=tflearn.DNN(net,tensorboard_dir='tflearn_logs')

# start training
# more number of epochs more accuarcy, show_metric for showing the performancereprot
model.fit(trainx,trainy,n_epoch=1000,batch_size=8,show_metric=True)
# saving the model so that no need to train again and again
# but if new intents are provided then we have to retrain the model
model.save('model.tflearn')



Training Step: 3999  | total loss: 0.02105 | time: 0.038s
| Adam | epoch: 1000 | loss: 0.02105 - acc: 1.0000 -- iter: 24/31
Training Step: 4000  | total loss: 0.02054 | time: 0.049s
| Adam | epoch: 1000 | loss: 0.02054 - acc: 1.0000 -- iter: 31/31
--


In [14]:
import pickle
#saving the model and intermediate data structrue
pickle.dump({'words':words,'classes':classes,'trainx':trainx,'trainy':trainy},open("training_data","wb"))

In [23]:
pickle.dump(model,open("trainedmodel.pkl",'wb'))

TypeError: ignored

In [15]:
# restoring the all data structrues
data=pickle.load(open('training_data','rb'))
words=data['words']
classes=data['classes']
trainx=data['trainx']
trainy=data['trainy']

In [16]:
with open('intents.json') as json_data:
  intents=json.load(json_data)

In [17]:
# loading the saved model
model.load('./model.tflearn')

In [18]:
# we will take inputs from user
# preprocess it into bag of words
def clean_sentences(sentence):
  # tokening the pattern
  sentence_words=nltk.word_tokenize(sentence)
  # stemming each word
  sentence_words=[stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words


# returning the bag of words array as 0/1 for each word in bag
def bow(sentence,words,show_details=False):
  # tokeninz the pattern
  sentence_words=clean_sentences(sentence)
  # generating bag of words
  bag=[0]*len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w==s:
        bag[i]=1
        if show_details:
          print("found in abd: %s",w)
  return (np.array(bag))


**Response** **Processor**

In [19]:
# # WITHOUT CONTEXT

# Error_threshold=0.30
# def classify(sentence):
#   # generate probabilities from model
#   results=model.predict([bow(sentence,words)])[0]
#   # filter out rpedictions below a threshold
#   results=[[i,r] for i,r in enumerate(results) if r>Error_threshold]
#   # sorting the strength of probabilties
#   results.sort(key=lambda x:x[1],reverse=True)
#   returnlist=[]
#   for r in results:
#     returnlist.append((classes[r[0]],r[1]))
#   # return tuple of intent and probability 
#   return returnlist


# # here userID is the context about which conversations
# def response(sentence, userID='123',show_details=False):
#   results=classify(sentence)
#   # if we hahve a classification then find the best mathcng intent tag
#   if results:
#     # loop as long as there are matches to process
#     while results:
#       for i in intents['intents']:
#         # find  a tag matching the first result
#         if i['tag']==results[0][0]:
#           # a random respose from the intent
#           return print(random.choice(i['responses']))
#       results.pop(0)

In [20]:
# # lets ask questions from chatbot
# classify('Bye')
# response('Bye')

In [21]:
# WITH CONTEXT

context={}
Error_threshold=0.30
def classify(sentence):
  # generate probabilities from model
  results=model.predict([bow(sentence,words)])[0]
  # filter out rpedictions below a threshold
  results=[[i,r] for i,r in enumerate(results) if r>Error_threshold]
  # sorting the strength of probabilties
  results.sort(key=lambda x:x[1],reverse=True)
  returnlist=[]
  for r in results:
    returnlist.append((classes[r[0]],r[1]))
  # return tuple of intent and probability 
  return returnlist


# here userID is the context about which conversations
def response(sentence, userID='123',show_details=False):
  results=classify(sentence)
  # if we hahve a classification then find the best mathcng intent tag
  if results:
    # loop as long as there are matches to process
    while results:
      for i in intents['intents']:
        # find  a tag matching the first result
        if i['tag']==results[0][0]:
          #set context for this intent if necessary
          if 'context_set' in i:
            if show_details:print('context:',i['context_set'])
            context[userID]=i['context_set']
          
          # check if this intent is contextual and applies to users conversation
          if not 'context_filter' in i or \
          (userID in context and 'context_filter' in i and i['context_filter']==context):
            if show_details:print('tag:',i['tag'])
            # a random respose from the intent
            return print(random.choice(i['responses']))
      results.pop(0)


In [22]:
response("bye")

Bye! Come back again soon.
